In [1]:
import imageio
import numpy as np
import numpy
from scipy import signal
from numpy.lib.stride_tricks import as_strided as ast
import glob
import tqdm
import os
from multiprocessing import Pool
import matplotlib.pyplot as plt
import random
import albumentations as A

def fspecial_gauss(size, sigma):
    """
        Function to mimic the 'fspecial' gaussian MATLAB function
    """
    x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g/g.sum()

def ssim(img1, img2, cs_map=False):
    """Return the Structural Similarity Map corresponding to input images img1 
    and img2 (images are assumed to be uint8)
    
    This function attempts to mimic precisely the functionality of ssim.m a 
    MATLAB provided by the author's of SSIM
    https://ece.uwaterloo.ca/~z70wang/research/ssim/ssim_index.m
    """
    #range [0,1]
    img1 = img1.astype(numpy.float64).clip(0, 8000.) / 8000.
    img2 = img2.astype(numpy.float64).clip(0, 8000.) / 8000.
    size = 11
    sigma = 1.5
    window = fspecial_gauss(size, sigma)
    K1 = 0.01
    K2 = 0.03
    L = 1. #bitdepth of image
    C1 = (K1*L)**2
    C2 = (K2*L)**2
    mu1 = signal.fftconvolve(window, img1, mode='valid')
    mu2 = signal.fftconvolve(window, img2, mode='valid')
    mu1_sq = mu1*mu1
    mu2_sq = mu2*mu2
    mu1_mu2 = mu1*mu2
    sigma1_sq = signal.fftconvolve(window, img1*img1, mode='valid') - mu1_sq
    sigma2_sq = signal.fftconvolve(window, img2*img2, mode='valid') - mu2_sq
    sigma12 = signal.fftconvolve(window, img1*img2, mode='valid') - mu1_mu2
    if cs_map:
        return (((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*
                    (sigma1_sq + sigma2_sq + C2)), 
                (2.0*sigma12 + C2)/(sigma1_sq + sigma2_sq + C2))
    else:
        return ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*
                    (sigma1_sq + sigma2_sq + C2))
    
def norm_shape(shape):
    '''
    Normalize numpy array shapes so they're always expressed as a tuple, 
    even for one-dimensional shapes.

    Parameters
        shape - an int, or a tuple of ints

    Returns
        a shape tuple
    '''
    try:
        i = int(shape)
        return (i,)
    except TypeError:
        # shape was not a number
        pass

    try:
        t = tuple(shape)
        return t
    except TypeError:
        # shape was not iterable
        pass

    raise TypeError('shape must be an int, or a tuple of ints')


def sliding_window(a,ws,ss = None,flatten = False):
    '''
    Return a sliding window over a in any number of dimensions

    Parameters:
        a  - an n-dimensional numpy array
        ws - an int (a is 1D) or tuple (a is 2D or greater) representing the size 
             of each dimension of the window
        ss - an int (a is 1D) or tuple (a is 2D or greater) representing the 
             amount to slide the window in each dimension. If not specified, it
             defaults to ws.
        flatten - if True, all slices are flattened, otherwise, there is an 
                  extra dimension for each dimension of the input.

    Returns
        an array containing each n-dimensional window from a

    from http://www.johnvinyard.com/blog/?p=268
    '''

    if None is ss:
        # ss was not provided. the windows will not overlap in any direction.
        ss = ws
    ws = norm_shape(ws)
    ss = norm_shape(ss)

    # convert ws, ss, and a.shape to numpy arrays so that we can do math in every 
    # dimension at once.
    ws = np.array(ws)
    ss = np.array(ss)
    shape = np.array(a.shape)


    # ensure that ws, ss, and a.shape all have the same number of dimensions
    ls = [len(shape),len(ws),len(ss)]
    if 1 != len(set(ls)):
        raise ValueError(\
        'a.shape, ws and ss must all have the same length. They were %s' % str(ls))

    # ensure that ws is smaller than a in every dimension
    if np.any(ws > shape):
        raise ValueError('ws cannot be larger than a in any dimension. a.shape was %s and ws was %s' % (str(a.shape),str(ws)))

    # how many slices will there be in each dimension?
    newshape = norm_shape(((shape - ws) // ss) + 1)
    # the shape of the strided array will be the number of slices in each dimension
    # plus the shape of the window (tuple addition)
    newshape += norm_shape(ws)
    # the strides tuple will be the array's strides multiplied by step size, plus
    # the array's strides (tuple addition)
    newstrides = norm_shape(np.array(a.strides) * ss) + a.strides
    strided = ast(a,shape = newshape,strides = newstrides)
    if not flatten:
        return strided

    # Collapse strided so that it has one more dimension than the window.  I.e.,
    # the new array is a flat list of slices.
    meat = len(ws) if ws.shape else 0
    firstdim = (np.product(newshape[:-meat]),) if ws.shape else ()
    dim = firstdim + (newshape[-meat:])
    # remove any dimensions with size 1
    dim = filter(lambda i : i != 1,dim)
    return strided.reshape(dim)

In [12]:
import imageio
a = imageio.imread('/all_data/Scannet_all_data/img/scene0000_00_0.jpg')
a.shape

(968, 1296, 3)

In [6]:
def calc_patches(fname):
    #resize
    resize = A.Compose([A.Resize(height=960, width=1280, interpolation=4, p=1)], p=1)
    fname_base, ext = os.path.splitext(fname)
    img = imageio.imread(os.path.join(sourse_path, 'img', fname_base + '.jpg'))
    img = resize(image=img)['image']
    depth_lq = imageio.imread(os.path.join(sourse_path, 'raw', fname))
    depth_sr = imageio.imread(os.path.join(sourse_path, 'render', fname))
    H, W = depth_lq.shape

    ssim_mtrx = ssim(depth_lq, depth_sr[0::2, 0::2])
    H_p, W_p = ssim_mtrx.shape
    assert ((H - H_p) % 2 == 0) and ((H - H_p) // 2 == (W - W_p) // 2 ), 'check how fftconvolve produce valid mode'
    pad = (H - H_p) // 2
    #valid mode don't return values influenced by zero-padding
    depth_sr = depth_sr[2*pad:-2*pad, 2*pad:-2*pad]
    depth_lq = depth_lq[pad:-pad, pad:-pad]
    img = img[2*pad:-2*pad, 2*pad:-2*pad, :]
    
    ssim_patch = sliding_window(ssim_mtrx, (320, 320), (64, 64))
    ssim_idx = ssim_patch.mean(axis=(2,3)) > 0.8
    n = ssim_idx.sum()

    if n > 0:
        
        depth_sr_patch = sliding_window(depth_sr, (640,640), (128,128))

        hole_v = 5
        hole_idx = (depth_sr_patch > hole_v).mean(axis=(2,3)) > 0.9
        final_idx = hole_idx *  ssim_idx
        n_good = final_idx.sum()
        patch_idx = np.argwhere(final_idx)
        if n_good == 0:
            return
        else:
            depth_lq_patch = sliding_window(depth_lq, (320, 320), (64, 64))
            img_patch = sliding_window(img, (640,640,3), (128, 128, 1) )
            
            depth_gt_good = depth_sr_patch[final_idx][:, 0::2, 0::2]
            depth_lq_good = depth_lq_patch[final_idx]
            img_good = img_patch[final_idx]
            depth_sr_good = depth_sr_patch[final_idx]
            for i in range(n_good):
                imageio.imsave("/all_data/hdd/un_depth/Scannet_filtered/img/{}_{}_{}{}".format(fname_base, patch_idx[i,0], patch_idx[i,1], ".jpg"), img_good[i,0])
                imageio.imsave("/all_data/hdd/un_depth/Scannet_filtered/raw/{}_{}_{}{}".format(fname_base, patch_idx[i,0], patch_idx[i,1], ext), depth_lq_good[i])
                imageio.imsave("/all_data/hdd/un_depth/Scannet_filtered/render/{}_{}_{}{}".format(fname_base, patch_idx[i,0], patch_idx[i,1], ext), depth_gt_good[i])
                imageio.imsave("/all_data/hdd/un_depth/Scannet_filtered/depth_sr/{}_{}_{}{}".format(fname_base, patch_idx[i,0], patch_idx[i,1], ext), depth_sr_good[i])
                

In [ ]:
sourse_path = '/all_data/hdd/un_depth/Scannet_all_data/' 
fnames = sorted(os.listdir(os.path.join(sourse_path, 'raw')))
n_processes = 15
with Pool(n_processes) as p:   
    res = list(tqdm.tqdm(p.imap(func=calc_ssim, iterable=fnames), total=len(fnames)))

# Check and count scene

In [34]:
path = '/all_data/hdd/un_depth/Scannet_filtered/'
test_path = '/all_data/hdd/un_depth/Scannet_filtered/test/'
val_path = '/all_data/hdd/un_depth/Scannet_filtered/val/'
img = sorted(os.listdir(os.path.join(path,'img')))
raw = sorted(os.listdir(os.path.join(path,'raw')))
render = sorted(os.listdir(os.path.join(path,'render')))
depth_sr = sorted(os.listdir(os.path.join(path,'depth_sr')))

In [35]:
len(img) == len(render) == len(raw) == len(depth_sr)

True

In [36]:
len(img)

126475

In [37]:
for i in range(len(img)):
    assert img[i].split('.')[0] == raw[i].split('.')[0] == render[i].split('.')[0] == depth_sr[i].split('.')[0]

In [38]:
count = {}
for i in range(len(img)):
    count[img[i].split("_")[0]] = count.get(img[i].split("_")[0], 0) + 1

In [45]:
# same_values = []
# for f in tqdm.tqdm(raw):
#     if np.unique(imageio.imread(os.path.join(path, 'raw', f))).shape[0] < 2 or np.unique(imageio.imread(os.path.join(path, 'render', f))).shape[0] < 2:
#         same_values.append(f)

In [40]:
len(count)

677

# Split val, test

In [41]:
n_test = 0
test_scene = []
while n_test < 10000:
    scene, n = random.choice(list(count.items()))
    n_test += n
    test_scene.append(scene)
    del count[scene]

In [42]:
len(test_scene)

59

In [47]:
len(count)

618

In [46]:
for f in tqdm.tqdm(img):
    if f.split("_")[0] in test_scene:
        name = f.split('.')[0]
        os.rename(os.path.join(path, 'img', name+'.jpg'), os.path.join(test_path, 'img', name+'.jpg'))
        os.rename(os.path.join(path, 'raw', name+'.png'), os.path.join(test_path, 'raw', name+'.png'))
        os.rename(os.path.join(path, 'render', name+'.png'), os.path.join(test_path, 'render', name+'.png'))
        os.rename(os.path.join(path, 'depth_sr', name+'.png'), os.path.join(test_path, 'depth_sr', name+'.png'))

100%|██████████| 126475/126475 [00:01<00:00, 108864.10it/s]


In [49]:
n_val = 0
val_scene = []
while n_val < 5000:
    scene, n = random.choice(list(count.items()))
    n_val += n
    val_scene.append(scene)
    del count[scene]

In [50]:
len(val_scene)

30

In [54]:
for f in tqdm.tqdm(img):
    if f.split("_")[0] in val_scene:
        name = f.split('.')[0]
        os.rename(os.path.join(path, 'img', name+'.jpg'), os.path.join(val_path, 'img', name+'.jpg'))
        os.rename(os.path.join(path, 'raw', name+'.png'), os.path.join(val_path, 'raw', name+'.png'))
        os.rename(os.path.join(path, 'render', name+'.png'), os.path.join(val_path, 'render', name+'.png'))
        os.rename(os.path.join(path, 'depth_sr', name+'.png'), os.path.join(val_path, 'depth_sr', name+'.png'))

100%|██████████| 116095/116095 [00:00<00:00, 177832.30it/s]


In [57]:
test_scene = np.array(test_scene)
val_scene = np.array(val_scene)
np.save('./test_scene', test_scene)
np.save('./val_scene', val_scene)

# Split train

In [61]:
img = sorted(os.listdir(os.path.join(path,'img')))

In [64]:
count = {}
for i in range(len(img)):
    count[img[i].split("_")[0]] = count.get(img[i].split("_")[0], 0) + 1

In [65]:
len(count)

588

In [67]:
n_trainA = 0
trainA_scene = []
while n_trainA < len(img) // 2:
    scene, n = random.choice(list(count.items()))
    n_trainA += n
    trainA_scene.append(scene)
    del count[scene]

In [70]:
trainA_path = '/all_data/hdd/un_depth/Scannet_filtered/trainA/'

In [71]:
for f in tqdm.tqdm(img):
    if f.split("_")[0] in trainA_scene:
        name = f.split('.')[0]
        os.rename(os.path.join(path, 'img', name+'.jpg'), os.path.join(trainA_path, 'img', name+'.jpg'))
        os.rename(os.path.join(path, 'raw', name+'.png'), os.path.join(trainA_path, 'raw', name+'.png'))
        os.rename(os.path.join(path, 'render', name+'.png'), os.path.join(trainA_path, 'render', name+'.png'))
        os.rename(os.path.join(path, 'depth_sr', name+'.png'), os.path.join(trainA_path, 'depth_sr', name+'.png'))

100%|██████████| 110749/110749 [00:06<00:00, 18190.60it/s]


In [72]:
n_trainA

56227